## 0. Simulate some data and fit an emulator

In [ ]:
import torch

from autoemulate.experimental.simulations.projectile import ProjectileMultioutput
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)

In [ ]:
param_ranges = {"c": (-1.0, 1.0), "v0": (20.0, 100), "angle": (30.0, 60.0), "h0": (0.0, 10.0)}
sim = ProjectileMultioutput()
x = sim.sample_inputs(1000)
y = sim.forward_batch(x)


In [ ]:
gp = GaussianProcessExact(x, y)
gp.fit(x, y)

In [ ]:
import time

# Create test parameters WITHOUT in-place operations
param_values = []
for param in param_ranges.keys():
    min_val, max_val = param_ranges[param]
    param_values.append((min_val + max_val) / 2)

test_params = torch.tensor([param_values], requires_grad=True)

# Time forward pass
start = time.time()
output = gp.predict(test_params)
forward_time = time.time() - start

# Time backward pass (gradient computation)
start = time.time()
if hasattr(output, 'sum'):
    loss = output.sum()
    loss.backward()
backward_time = time.time() - start

print(f"Forward pass: {forward_time:.4f}s")
print(f"Backward pass: {backward_time:.4f}s")

## 1. Simple HMC example.

In [ ]:
from autoemulate.experimental.calibration.hmc import MCMC_calibration

Start with an "observation" that the GP has been trained on. 

Specifically, we will pretend we have N noisy experimental measurements. We should be able to recover the input parameters.

In [ ]:
idx = -1 # which simulated value to pick
n_obs = 50
noise_scale = 0.05 # set noise as some ratio of the observed value

observations = {
    "distance": y[idx, 0].repeat(n_obs) + torch.rand(n_obs) * noise_scale * y[idx, 0], 
    "impact_velocity": y[idx, 1].repeat(n_obs) + torch.rand(n_obs) * noise_scale * y[idx, 1]
}
observations

In [ ]:
# Test emulator speed
import time
parameter_range = {"c": (-1.0, 0.0), "v0": (20.0, 60), "angle": (30.0, 60.0), "h0": (0.0, 5.0)}

test_params = torch.randn(100, len(parameter_range))
start = time.time()
predictions = gp.predict(test_params)
print(f"100 predictions took: {time.time() - start:.3f}s")

In [ ]:
# use the simulator parameter_range 
hmc = MCMC_calibration(gp, sim.parameters_range, observations, 10.0)

Run MCMC (note that below we have set the number of MCMC steps to a very low number, don't expect convergence).

In [ ]:
mcmc = hmc.run(
    warmup_steps=100, 
    num_samples=100,
    sampler='nuts',  # or 'nuts' or 'hmc'
    # also init with x values matching "observations"
    )

The returned Pyro MCMC object has methods for accessing the generated samples (`mcmc.get_samples()`) or, as shown below, to get their summary statistics.

In [ ]:
mcmc.summary()

## 2. Plotting with Arviz

We have an option to turn the MCMC object into an Arviz object, which can be passed to any of their plotting function.

In [ ]:
import arviz as az

In [ ]:
az_data = hmc.to_arviz(mcmc, posterior_predictive=True)

In [ ]:
az.plot_trace(az_data)

In [ ]:
az.plot_pair(az_data, kind='kde')

In [ ]:
az.plot_ppc(az_data, kind='scatter')

In [ ]:
az.plot_autocorr(az_data)

## 3. Use sensitivity analysis and history matching to refine problem before running HMC.

The `MCMC_calibration` object has an option to provide a list of parameters to calibrate. A common approach is to select these based on results of `SensitivityAnalysis`.

Similarly, the user provides parameter ranges from withing which to sample parameter values. This can be simply the range of the simulator. Alternatively, one can use `HistoryMatching` to reduce the parameter range and pass that to the `MCMC_calibration` instead. 

Below we demonstrate how to do both.

In [ ]:
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis
from autoemulate.experimental.calibration.history_matching import HistoryMatching

1. Run sensitivity analysis and get top N parameters (here we just get the top 1).

In [ ]:
problem = {
        "num_vars": 2,
        "names": sim.param_names,
        "bounds": sim.param_bounds,
    }

sa = SensitivityAnalysis(gp, problem=problem)
df = sa.run("sobol")

top_param = sa.top_n_sobol_params(df, 1)

# the output is just a list of strings, this could be set by hand
top_param

2. Run history matching and generate new parameter bounds from NROY samples (if get any).

In [ ]:
# start with some GP predictions
x_new = sim.sample_inputs(20)
output = gp.predict(torch.tensor(x_new, dtype=torch.float32))
pred_means, pred_vars = (
    output.mean.float().detach(),
    output.variance.float().detach(),
)

In [ ]:
# generate NROY samples
hm = HistoryMatching(
    # take mean of observations and add noise
    observations={k: [v.mean(), 10.0] for k,v in observations.items()},
    threshold=5.0,
    rank=2
)
implausability = hm.calculate_implausibility(pred_means, pred_vars)
nroy_samples = hm.get_nroy(implausability, x_new)
nroy_samples

The newly generated range is slightly narrower than the range of the simulator.

In [ ]:
# get new param bounds
nroy_param_range = hm.generate_param_bounds(nroy_samples, param_names = sim.param_names)
nroy_param_range

3. Pass results to the `MCMC_calibration` object.

In [ ]:
hmc_nroy = MCMC_calibration(
    gp, 
    nroy_param_range if nroy_param_range is not None else sim.parameters_range, 
    observations, 
    10.0,
    top_param
    )

In [ ]:
mcmc_nroy = hmc_nroy.run(warmup_steps=10, num_samples=100)

In [ ]:
mcmc_nroy.summary()